# Simulating Delta Hedging Strategies in the Black-Scholes Model

> *Author: Amos Anderson*  
> *Program: Applied Mathematics & Statistics (Quantitative Finance)*  
> *University: Stony Brook University*  

## Introduction

Managing financial risk requires more than just intuition—it requires models, mathematics, and adaptive strategies. In this project, we take a deep dive into the mechanics of **delta hedging** within the **Black-Scholes framework**, focusing on an European call option. Delta hedging involves continuously adjusting a position in the underlying asset to offset the changing sensitivity (delta) of an option, thereby neutralizing exposure to small price movements.

We begin by pricing the option using the closed-form Black-Scholes formula, then compute its delta to construct an initial hedged portfolio. As the stock price evolves over two days, we simulate the rebalancing actions an option writer must take to maintain a delta-neutral position. Through this, we quantify the profit and loss dynamics of hedging in response to market changes.

This project is structured as a Jupyter Notebook that integrates precise mathematical derivations with clean, object-oriented Python code. It features a modular `BlackScholes` class that handles pricing, Greeks computation, and portfolio simulation. By combining theory with hands-on implementation, we aim to provide a clear and replicable framework for understanding how derivative pricing translates into real-world trading strategy.


## Project Structure
* Problem Description: Details of the option pricing and hedging scenario.
* Mathematical Foundations: Derivations of the Black-Scholes model and delta hedging.
* Python Implementation: A BlackScholes class and simulation code.
* Results: Outputs for pricing, delta, and hedging outcomes.
* Conclusion: Reflections on the project and its applicability.

## Problem Description:

We consider a European call option in a Black-Scholes setting with the following parameters:

- **Stock price**  ($S_0$): \$35 per share  
- **Strike price**  ($K$): \$33  
- **Volatility**  ($\sigma$): 25% (0.25)  
- **Dividend yield**  ($D$): 2% (0.02)  
- **Risk-free rate**  ($r$): 5% (0.05) per annum  
- **Time to expiration** ($T$): 180 days  
- **Shares underlying the option**: 1000  

Our objectives are to:

- Price the option and compute its delta at the initial time ($t=0$).
- Design a delta-hedging strategy to neutralize market risk, determining the shares to buy and the amount to borrow or invest.
- Simulate hedging on day 1 when the stock price rises to \$35.50, calculating the profit/loss and rebalancing cost.
- Simulate hedging on day 2 when the stock price falls to \$34.80, calculating the profit/loss and rebalancing cost.


## Mathematical Foundations

### Black-Scholes Call Price

The Black-Scholes price for a European call option with continuous dividends is given by:

$$
C = S_t e^{-D(T - t)} \Phi(d_1) - K e^{-r(T - t)} \Phi(d_2)
$$

where:

$$
d_1 = \frac{\log\left(\frac{S_t}{K}\right) + \left(r - D + \frac{1}{2} \sigma^2\right)(T - t)}{\sigma \sqrt{T - t}}
$$

$$
d_2 = d_1 - \sigma \sqrt{T - t}
$$

and:

$$
\Phi(x) = \mathbb{P}(Z \leq x), \quad Z \sim \mathcal{N}(0, 1)
$$

---

### Approximation of the Standard Normal CDF

For $x > 0$, we use the following approximation:

$$
\Phi(x) \approx 0.5 + \frac{1}{\sqrt{2\pi}} \left(x - \frac{x^3}{6} + \frac{x^5}{40} - \frac{x^7}{336} + \frac{x^9}{3456}\right)
$$

For $x < 0$, the identity:

$$
\Phi(x) = 1 - \Phi(-x)
$$

is used to compute the value based on its positive counterpart.

### Total Option Value

The total option value for 1000 shares is:

$$
\text{Total Option Value} = 1000 \times C
$$

---

### Delta

The delta, or sensitivity of the option price to the stock price, is given by:

$$
\Delta = \frac{\partial C}{\partial S} = e^{-D(T - t)} \Phi(d_1)
$$

For 1000 shares, the total delta is:

$$
\text{Total Delta} = 1000 \times \Delta
$$

---

### Delta-Hedging Strategy

The option writer sells the call, receiving the premium, and hedges by holding shares to offset the option’s delta. The portfolio’s delta is:

$$
\Delta_{\text{portfolio}} = \Delta_{\text{shares}} + \Delta_{\text{option}}
$$

Since the writer is **short** the option, the option’s delta is:

$$
\Delta_{\text{option}} = -1000 \times \Delta
$$

To achieve **delta neutrality**, the writer buys:

$$
\text{Shares} = 1000 \times \Delta
$$

at a cost of:

$$
\text{Cost} = 1000 \times \Delta \times S_t
$$

The net cash position is:

$$
\text{Net Cash} = \text{Premium} - \text{Cost}
$$

If this value is negative, the writer borrows at rate $r$; if positive, they invest the surplus.

---

### Profit and Rebalancing

At each day, the stock price changes, requiring a new delta calculation. The portfolio value becomes:

$$
\text{Portfolio Value} = (\text{Shares Held}) \times S_t - (\text{Option Value}) - (\text{Debt with Interest})
$$

**Profit** is the change in portfolio value relative to the initial value (typically zero for a self-financing hedge).

To **rebalance**, the writer adjusts their position to the new delta by buying or selling shares at the current market price, and updates the debt or cash accordingly.

## Python Implementation
The following Python code implements the Black-Scholes model and delta-hedging simulation. The BlackScholes class provides methods for pricing and delta calculations, using the provided CDF approximation. The simulation computes results for the initial setup and two-day hedging scenario.

In [23]:
import numpy as np
from scipy.stats import norm
import math

class BlackScholes:
    """
    A scikit-learn-style class for Black-Scholes option pricing and delta calculations.
    
    Parameters
    ----------
    S : float
        Current stock price.
    K : float
        Strike price.
    T : float
        Time to expiration (in years).
    r : float
        Risk-free interest rate (annualized).
    sigma : float
        Volatility (annualized).
    D : float, optional
        Dividend yield (annualized). Default is 0.0.
    
    Methods
    -------
    compute_d1_d2()
        Compute d1 and d2 parameters.
    norm_cdf_approx(x)
        Approximate standard normal CDF using the provided polynomial.
    call_price()
        Compute the Black-Scholes call option price per share.
    call_delta()
        Compute the call option delta per share.
    """
    
    def __init__(self, S, K, T, r, sigma, D=0.0):
        self.S = float(S)
        self.K = float(K)
        self.T = float(T)
        self.r = float(r)
        self.sigma = float(sigma)
        self.D = float(D)
        
        # Input validation
        if any(param <= 0 for param in [S, K, T, sigma]):
            raise ValueError("S, K, T, and sigma must be positive.")
        if r < 0 or D < 0:
            raise ValueError("r and D must be non-negative.")
    
    def compute_d1_d2(self):
        """Compute d1 and d2 parameters for Black-Scholes."""
        log_term = math.log(self.S / self.K)
        drift = (self.r - self.D + 0.5 * self.sigma**2) * self.T
        denominator = self.sigma * math.sqrt(self.T)
        d1 = (log_term + drift) / denominator
        d2 = d1 - self.sigma * math.sqrt(self.T)
        return d1, d2
    
    def norm_cdf_approx(self, x):
        """
        Approximate standard normal CDF for x using the provided polynomial for 0 < x <= 1.
        For x < 0, use Phi(-x) = 1 - Phi(x). For x > 1, use scipy.norm.cdf.
        
        Parameters
        ----------
        x : float
            Input value for CDF.
        
        Returns
        -------
        float
            Approximated CDF value.
        """
        if x < 0:
            return 1 - self.norm_cdf_approx(-x)
        if x > 1:
            return norm.cdf(x)  # Use scipy for accuracy beyond approximation range
        c = 1 / math.sqrt(2 * math.pi)
        poly = x - (x**3) / 6 + (x**5) / 40 - (x**7) / 336 + (x**9) / 3456
        return 0.5 + c * poly
    
    def call_price(self):
        """Compute Black-Scholes call option price per share."""
        d1, d2 = self.compute_d1_d2()
        phi_d1 = self.norm_cdf_approx(d1)
        phi_d2 = self.norm_cdf_approx(d2)
        term1 = self.S * math.exp(-self.D * self.T) * phi_d1
        term2 = self.K * math.exp(-self.r * self.T) * phi_d2
        return term1 - term2
    
    def call_delta(self):
        """Compute call option delta per share."""
        d1, _ = self.compute_d1_d2()
        phi_d1 = self.norm_cdf_approx(d1)
        return math.exp(-self.D * self.T) * phi_d1

def delta_hedge_simulation():
    """
    Simulate delta hedging for a European call option over two days.
    
    Returns
    -------
    dict
        Results for initial pricing, hedging strategy, and two-day simulation.
    """
    # Initial parameters
    S0 = 35.0  # Initial stock price
    K = 33.0   # Strike price
    T0 = 180 / 365  # Initial time to expiration (years)
    r = 0.05   # Risk-free rate
    sigma = 0.25  # Volatility
    D = 0.02   # Dividend yield
    N = 1000   # Number of shares underlying the option
    
    # Day 0: Initial pricing and delta
    bs_day0 = BlackScholes(S=S0, K=K, T=T0, r=r, sigma=sigma, D=D)
    call_price_day0 = bs_day0.call_price() * N
    delta_day0 = bs_day0.call_delta() * N
    
    # Day 0: Hedging strategy
    shares_day0 = delta_day0
    cost_day0 = shares_day0 * S0
    premium = call_price_day0
    borrowing_day0 = cost_day0 - premium  # Borrow if positive
    
    # Day 1: Stock price rises to 35.50
    S1 = 35.50
    T1 = (180 - 1) / 365
    bs_day1 = BlackScholes(S=S1, K=K, T=T1, r=r, sigma=sigma, D=D)
    call_price_day1 = bs_day1.call_price() * N
    delta_day1 = bs_day1.call_delta() * N
    # Rebalancing
    shares_to_buy_day1 = delta_day1 - shares_day0
    cost_day1 = shares_to_buy_day1 * S1
    # Portfolio value
    debt_day1 = borrowing_day0 * math.exp(r * 1 / 365)
    portfolio_value_day1 = (shares_day0 * S1) - call_price_day1 - debt_day1
    profit_day1 = portfolio_value_day1  # Initial portfolio value is 0
    
    # Day 2: Stock price falls to 34.80
    S2 = 34.80
    T2 = (180 - 2) / 365
    # Assume rebalanced at end of day 1
    current_shares = delta_day1
    current_debt = debt_day1 + cost_day1
    bs_day2 = BlackScholes(S=S2, K=K, T=T2, r=r, sigma=sigma, D=D)
    call_price_day2 = bs_day2.call_price() * N
    delta_day2 = bs_day2.call_delta() * N
    # Rebalancing
    shares_to_adjust_day2 = delta_day2 - current_shares
    cost_day2 = shares_to_adjust_day2 * S2
    # Portfolio value
    debt_day2 = current_debt * math.exp(r * 1 / 365)
    portfolio_value_day2 = (current_shares * S2) - call_price_day2 - debt_day2
    profit_day2 = portfolio_value_day2
    
    # Compile results
    results = {
        'initial': {
            'call_price': call_price_day0,
            'delta': delta_day0
        },
        'hedge_setup': {
            'shares': shares_day0,
            'borrowing': borrowing_day0
        },
        'day1': {
            'profit': profit_day1,
            'rebalancing_cost': cost_day1
        },
        'day2': {
            'profit': profit_day2,
            'rebalancing_cost': cost_day2
        }
    }
    return results

def main():
    """Execute the delta hedging simulation and display results."""
    results = delta_hedge_simulation()
    
    print("Black-Scholes Delta Hedging Simulation Results")
    print("===============================================")
    print(f"Initial Setup:")
    print(f"  Call Option Value (1000 shares): ${results['initial']['call_price']:.2f}")
    print(f"  Delta (1000 shares): {results['initial']['delta']:.3f} shares")
    print(f"Hedging Strategy:")
    print(f"  Shares Purchased: {results['hedge_setup']['shares']:.3f}")
    print(f"  Amount Borrowed: ${results['hedge_setup']['borrowing']:.2f}")
    print(f"Day 1 (Stock Price = $35.50):")
    print(f"  Profit/Loss: ${results['day1']['profit']:.2f}")
    print(f"  Rebalancing Cost: ${results['day1']['rebalancing_cost']:.2f}")
    print(f"Day 2 (Stock Price = $34.80):")
    print(f"  Profit/Loss: ${results['day2']['profit']:.2f}")
    print(f"  Rebalancing Cost: ${results['day2']['rebalancing_cost']:.2f}")

if __name__ == "__main__":
    main()

Black-Scholes Delta Hedging Simulation Results
Initial Setup:
  Call Option Value (1000 shares): $3770.33
  Delta (1000 shares): 687.184 shares
Hedging Strategy:
  Shares Purchased: 687.184
  Amount Borrowed: $20281.10
Day 1 (Stock Price = $35.50):
  Profit/Loss: $-2.43
  Rebalancing Cost: $984.52
Day 2 (Stock Price = $34.80):
  Profit/Loss: $-11.36
  Rebalancing Cost: $-1351.27


## Results

### Initial Setup

- **Call Option Value (1000 shares)**: \$3,770.33  
- **Initial Delta (1000 shares)**: 687.184 shares  
- **Shares Purchased to Hedge**: 687.184  
- **Amount Borrowed**: \$20,281.10  

At time zero, the option writer receives $\$ 3,770.33$ in premium for selling the European call option. To hedge the position, they compute the option's delta and purchase 687.184 shares of the underlying stock, which costs more than the premium. As a result, they borrow \$20,281.10 at the risk-free rate to finance the hedge.

---

### Day 1 — Stock Price = \$35.50

- **Profit/Loss**: \$−2.43  
- **Rebalancing Cost**: \$984.52  

As the stock price increases, the delta of the call option increases, meaning the hedge becomes underweighted. To restore delta neutrality, the writer purchases additional shares. This adjustment costs \$984.52 and is financed by increasing the debt. Despite the adjustment, the hedged portfolio records a small **loss of \$2.43**, reflecting minor inefficiencies due to discrete (not continuous) rebalancing.

---

### Day 2 — Stock Price = \$34.80

- **Profit/Loss**: \$−11.36  
- **Rebalancing Cost**: \$−1,351.27  

On Day 2, the stock price falls. The option's delta decreases, so the hedge is now overweighted. The writer sells a portion of their holdings, which generates \$1,351.27 in cash and reduces the debt. However, the overall **loss accumulates to \$11.36**. This loss captures the tracking error of the delta hedge due to the mismatch between the actual path of the stock and the assumptions of continuous trading and lognormal dynamics.

---

### Interpretation

These results highlight the **practical realities of delta hedging**:

- Even in a theoretically perfect Black-Scholes world, **discrete rebalancing introduces hedging error**.
- Small day-to-day changes in stock price cause small gains or losses in the hedged portfolio.
- The **rebalancing cost reflects how sensitive the hedge is to stock movements**, and these costs can accumulate over time.
- Importantly, the losses remain small relative to the size of the position, confirming that delta hedging **substantially reduces directional risk** even if it doesn’t eliminate all risk.

This simulation demonstrates how delta hedging behaves over time and provides insight into the trade-offs between **hedging accuracy**, **transaction costs**, and **portfolio management** under real-world constraints.

## Conclusion

This project demonstrates how the Black-Scholes model and delta-hedging principles can be applied to manage the risk of a European call option in a dynamic market environment. By calculating the option price and delta at inception, and simulating a hedged portfolio over two days of stock price movement, we explored the real-world implications of a theoretically sound strategy.

The results confirm that delta hedging effectively reduces directional exposure, even with discrete rebalancing. However, small hedging errors and rebalancing costs emerge due to the mismatch between continuous-time assumptions and real-world trading. These deviations, though minor, underscore the importance of frequent monitoring and adjustment in managing financial derivatives.

Ultimately, this project bridges theory and practice, providing a modular, transparent Python implementation that highlights the mechanics and trade-offs of hedging. It serves as a foundational tool for understanding how mathematical models translate into portfolio decisions—and offers a starting point for extending to more complex strategies like gamma hedging or stochastic volatility models.
